In [1]:
###Run this code to create a JSON file that diables auto-quotes and auto-brackets. After executing the Python 
#command, or manually creating the file, restart your Jupyter notebook, and it should stop auto-closing quotes
#and brackets.

#from notebook.services.config import ConfigManager
#c = ConfigManager()
#c.update('notebook', {"CodeCell": {"cm_config": {"autoCloseBrackets": False}}})

###Keyboard Shortcuts
#Ctrl + Enter: Run single cell of code (similar to R)
#Ctrl + Shift + Enter: Run entire notebook (similar to R)

In [2]:
#Intro to Web Scraping and BeautifulSoup
#https://www.youtube.com/watch?v=XQgXKtPSzUI

#Beautiful Soup HTTP Error 403
#URL = 'https://www.petflytrap.com/All-carnivorous-plants_c_11-%d-3.html' % page_number
#h = {'User-Agent':'Mozilla/5.0'}
#req = Request(URL,headers=h) 
#page = urlopen(req).read()
#soup = BeautifulSoup(page,'html.parser')

#exporting to excel
#df.to_excel("excel_file_name.xlsx")

import pandas as pd
import numpy as np
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
#import re #for splitting strings using multiple delimiters
import time
import random

names = []
prices = []
saleprices = []
soldouts = []
stores = []
species = []

In [3]:
#When populating species_XXX variables, make sure that they are all lowercase; to make comparison easier
#Common name on left side, taxonomy on right
species_vft = ['venus','flytrap','flytraps','dionaea','muscipula']
species_neps = ['tropical','asian','nepenthes','N.']
species_sarracenia = ['american','sarracenia']
species_sundew = ['sundew','sundews','drosera']
species_utric = ['bladderwort','bladderworts','utricularia']
species_pings = ['butterwort','butterworts','pinguicula']
species_cephs = ['australian','cephalotus']
species_darlingtonia = ['cobra','lily','lilies','darlingtonia']
species_heli = ['sun','heliamphora']
#species_special = ['specimen plants','carnivero exclusives']

species_all = species_vft + species_neps + species_sarracenia + species_sundew + species_utric \
            + species_pings + species_cephs + species_darlingtonia + species_heli
    
#function to check type of species
#name is the name of the item as a string, returns a string of the type of species
def check_species(name):
    #cleaning: replacing special characters with spaces, all lowercase then splitting words into list
    kind_list = name.strip().replace('(','').replace(')','').replace('[','').replace(']','').replace(',','')
    kind_list = kind_list.lower().split(' ')
    
    n = len(kind_list)
    if n == 0:
        print('error: length of kind is 0, cannot loop')
        return 'error'
    
    #will try each element in the kind list and see if it is in species_XXX
    #if not, will keep looping over the length of n
    for i in range(n):
        if kind_list[i] in species_vft:
            return 'dionaea muscipula'
        elif kind_list[i] in species_neps:
            return 'nepenthes'
        elif kind_list[i] in species_sarracenia:
            return 'sarracenia'    
        elif kind_list[i] in species_sundew:
            return 'drosera'
        elif kind_list[i] in species_pings:
            return 'pinguicula'
        elif kind_list[i] in species_utric:
            return 'utricularia'
        elif kind_list[i] in species_cephs:
            return 'cephalotus'
        elif kind_list[i] in species_darlingtonia:
            return 'darlingtonia'
        elif kind_list[i] in species_heli:
            return 'heliamphora'
        elif i == (n-1):
            #when list of strings are not found in species_XXX, return other
            return 'other'

In [4]:
[len(names),len(prices),len(saleprices),len(soldouts),len(stores),len(species)]

[0, 0, 0, 0, 0, 0]

In [5]:
"""
#last checked 3/28/20
names = []; prices = []; saleprices = []; soldouts = []; stores = []; species = []

#Pearl River Exotics
URL = 'https://www.pearlriverexotics.com'
page = urlopen(URL)
soup = BeautifulSoup(page,'html.parser')
links = soup.findAll('a',attrs={'class':'site-nav__link site-nav__link--main'})
links = links[:len(links)-5] #excluding last sections 'carnivorous plants for beginners' to 'shipping policy'

for link in links:
    #Create URL2 to be URL of species type
    path = link['href']
    URL2 = URL + path

    page = urlopen(URL2)
    soup = BeautifulSoup(page,'html.parser')
    #finds how many pages to loop over
    try:
        page_count = soup.find('li',attrs={'class':'pagination__text'}).text.strip()
        page_count = page_count[len(page_count)-1:] #grabs how many pages to loop through
        page_count = int(page_count)
    except (AttributeError,IndexError):
        page_count = 1 #only 1 page

    for page_number in range(page_count):
        #wait random time between 0-5 seconds before scraping data
        r = random.randint(0,5)
        time.sleep(r)
    
        page = urlopen(URL2)
        soup = BeautifulSoup(page,'html.parser')
        containers = soup.findAll('a',attrs={'class':'grid-view-item__link'})

        for container in containers:
            name = container.find('div',attrs={'class':'h4 grid-view-item__title'}).text.strip()
            specie = check_species(name)

            price = container.find('span',attrs={'class':'product-price__price'}).text.strip()
            #if sale is in price text, get price and sale price
            if 'Sale' in price:
                separate_index = price.find('\n') #from BEGINNING of string, look for 1st instance of '\n'
                saleprice = price[1:separate_index]
                
                price = container.find('s',attrs={'class':'product-price__price'}).text.strip()
                price = price[1:]
            else:
                price = price[1:] #remove the 1st character of the string '$'
                saleprice = ''
    
            names = names + [name]
            prices = prices + [price]
            stores = stores + ['Pearl River Exotics']
            saleprices = saleprices + [saleprice]
            species = species + [specie]
            
            #looks for sold out items
            try:
                soldout = container.find('span',attrs={'class':'product-price__sold-out'}).text.strip()
                soldouts = soldouts + [soldout.lower()]
            except AttributeError:
                soldouts = soldouts + ['']
        
        #once page is scraped, grabs the URL for next page
        try:
            path = soup.findAll('a',attrs={'class':'btn btn--secondary btn--narrow'})
            path = path[len(path)-1] #gets the link to the next page
            path = path['href']
            URL2 = URL + path
        except (AttributeError,IndexError):
            continue #nothing

d = {'Name':names,'Price':prices,'Sale Price':saleprices,'Sold Out':soldouts,'Store':stores,'Species':species}
df = pd.DataFrame(d)
df
"""

"\n#last checked 3/28/20\nnames = []; prices = []; saleprices = []; soldouts = []; stores = []; species = []\n\n#Pearl River Exotics\nURL = 'https://www.pearlriverexotics.com'\npage = urlopen(URL)\nsoup = BeautifulSoup(page,'html.parser')\nlinks = soup.findAll('a',attrs={'class':'site-nav__link site-nav__link--main'})\nlinks = links[:len(links)-5] #excluding last sections 'carnivorous plants for beginners' to 'shipping policy'\n\nfor link in links:\n    #Create URL2 to be URL of species type\n    path = link['href']\n    URL2 = URL + path\n\n    page = urlopen(URL2)\n    soup = BeautifulSoup(page,'html.parser')\n    #finds how many pages to loop over\n    try:\n        page_count = soup.find('li',attrs={'class':'pagination__text'}).text.strip()\n        page_count = page_count[len(page_count)-1:] #grabs how many pages to loop through\n        page_count = int(page_count)\n    except (AttributeError,IndexError):\n        page_count = 1 #only 1 page\n\n    for page_number in range(page_c

In [6]:
"""
#last checked 3/22/20; still need to check sold out case
names = []; prices = []; saleprices = []; soldouts = []; stores = []; species = []

#California Carnivores
URL = 'https://www.californiacarnivores.com/'
page = urlopen(URL)
soup = BeautifulSoup(page,'html.parser')
soup = soup.find('ul',attrs={'class':'sidebar-module__list'})
links = soup.findAll('a')
links = [links[i] for i in range(len(links)) if i not in (0,1,3,13,16,22,26,30,31,32,33,34,35,36,37)]
#exluding following pages: plant collections (0), easy to grow (1), drosera (3), sarracenia (13),
#nepenthes (16), pinguicula (22), utricularia (26), waterwheel, ..., gifts (30-37)

for link in links:
    #Create URL2 to be URL of species type
    path = link['href']
    URL2 = URL + path
    
    #wait random time between 0-5 seconds before scraping data
    r = random.randint(0,5)
    time.sleep(r)
    
    page_next = True
    while page_next:
        page = urlopen(URL2)
        soup = BeautifulSoup(page,'html.parser')
        containers = soup.findAll('div',attrs={'class':'grid__item large--one-quarter medium-down--one-half'})
        containers_sale = soup.findAll('div',attrs={'class':'grid__item large--one-quarter medium-down--one-half on-sale'})

        #grabs information of normal items
        for container in containers:
            name = container.find('p',attrs={'class':'grid-link__title'}).text.strip()
            specie = check_species(name)

            price = container.find('p',attrs={'class':'grid-link__meta'}).text.strip()
            separate_index = price.find('$') #look for 1st instance of $
            price = price[separate_index+2:] #remove the all characters before and including the string '$ '
    
            names = names + [name]
            prices = prices + [price]
            saleprices = saleprices + ['']
            stores = stores + ['California Carnivores']
            soldouts = soldouts + ['']
            species = species + [specie]
        
        #grabs information of sale items
        for container in containers_sale:
            name = container.find('p',attrs={'class':'grid-link__title'}).text.strip()
            specie = check_species(name)

            price = container.find('p',attrs={'class':'grid-link__meta'}).text.strip() #price of items on sale should be of form '$ xx.xx\n$ xx.xx'
            separate_index = price.rfind('$') #from end of string, look for 1st instance of $
            price_before = price[separate_index:] #gets original price
            price_before = price_before[2:] #remove the first 2 characters of the string '$ '
            price_after = price[:separate_index] #gets sale price
            price_after = price_after[2:] #remove the first 2 characters of the string '$ '
            price_after = price_after[:-1] #remove the last 2 characters of the string '$ '
    
            names = names + [name]
            prices = prices + [price_before]
            saleprices = saleprices + [price_after]
            stores = stores + ['California Carnivores']
            soldouts = soldouts + ['']
            species = species + [specie]
        
        #Updates the URL for next page; stops the while loop if there is no next page
        try:
            path = soup.find('a',attrs={'title':'Next »'})
            path = path['href']
            URL2 = URL + path
        except TypeError:
            page_next = False
        
d = {'Name':names,'Price':prices,'Sale Price':saleprices,'Sold Out':soldouts,'Store':stores,'Species':species}
df1 = pd.DataFrame(d)
df = df.append(df1,ignore_index=True)
df1
"""

"\n#last checked 3/22/20; still need to check sold out case\nnames = []; prices = []; saleprices = []; soldouts = []; stores = []; species = []\n\n#California Carnivores\nURL = 'https://www.californiacarnivores.com/'\npage = urlopen(URL)\nsoup = BeautifulSoup(page,'html.parser')\nsoup = soup.find('ul',attrs={'class':'sidebar-module__list'})\nlinks = soup.findAll('a')\nlinks = [links[i] for i in range(len(links)) if i not in (0,1,3,13,16,22,26,30,31,32,33,34,35,36,37)]\n#exluding following pages: plant collections (0), easy to grow (1), drosera (3), sarracenia (13),\n#nepenthes (16), pinguicula (22), utricularia (26), waterwheel, ..., gifts (30-37)\n\nfor link in links:\n    #Create URL2 to be URL of species type\n    path = link['href']\n    URL2 = URL + path\n    \n    #wait random time between 0-5 seconds before scraping data\n    r = random.randint(0,5)\n    time.sleep(r)\n    \n    page_next = True\n    while page_next:\n        page = urlopen(URL2)\n        soup = BeautifulSoup(pa

In [7]:
"""
#last checked 3/22/20; still need to check sold out case
names = []; prices = []; saleprices = []; soldouts = []; stores = []; species = []

#Cook's Carnivorous Plants
URL = 'http://www.flytraps.com/Scripts/'
page = urlopen(URL)
soup = BeautifulSoup(page,'html.parser')
links = soup.findAll('span',attrs={'class':'CPcatDescProd'})
links = links[:25] #from aldrovanda to nepenthes hybrid lowland

for link in links:
    #Checks what type of species
    kind = link.text
    specie = check_species(kind)
        
    #Create URL2 to be URL of species type
    path = link.a['href']
    URL2 = URL + path
    
    #wait random time between 0-5 seconds before scraping data
    r = random.randint(0,5)
    time.sleep(r)
    
    page = urlopen(URL2)
    soup = BeautifulSoup(page,'html.parser')
    containers = soup.findAll('div',attrs={'class':'prod-classic'})
    containers_count = len(containers)
    
    #print(URL2)
    #If is not blank, scrapes info. otherwise skips page
    if containers_count != 0:
        
        #each page contains 15 items, goes to next page when page_next mod 15 == 0
        page_next = 0
        while page_next % 15 == 0:
            page = urlopen(URL2)
            soup = BeautifulSoup(page,'html.parser')
            containers = soup.findAll('div',attrs={'class':'prod-classic'})
    
            for container in containers:
                name = container.h3.text.strip()
            
                #Most items on site are on sale. Try to process items as if the were on sale. If not on sale, then exception
                try:
                    price = container.find('del',attrs={'class':'CPprodLPriceV'}).text.strip()
                    price = price[1:] #removed '$' at beginning
            
                    saleprice = container.find('span',attrs={'class':'price'}).text.strip()
                    separate_index = saleprice.find('$') #look for 1st instance of $
                    saleprice = saleprice[separate_index+1:] #remove the all characters before and including the string '$'
                    saleprice = saleprice[:-3] #remove ' ea' from end of sales price
                except AttributeError:
                    price = container.find('span',attrs={'class':'price'}).text.strip()
                    separate_index = price.find('$') #look for 1st instance of $
                    price = price[separate_index+1:] #remove the all characters before and including the string '$'
                    price = price[:-3] #remove ' ea' from end of sales price
                
                    saleprice = ''
            
                names = names + [name]
                prices = prices + [price]
                stores = stores + ['Cook\'s Carnivorous Plants']
                soldouts = soldouts + ['']
                species = species + [specie]
                saleprices = saleprices + [saleprice]
                page_next = page_next + 1
                
            #Updates the URL for next page; stops the while loop if there is no next page
            if page_next % 15 == 0:
                paths = soup.find('div',attrs={'class':'spacing fl-right'})
                paths = paths.findAll('a')
                path = paths[len(paths)-1]['href'] #href of the last link (should be next page)
                URL2 = URL + path
        
d = {'Name':names,'Price':prices,'Sale Price':saleprices,'Sold Out':soldouts,'Store':stores,'Species':species}
df1 = pd.DataFrame(d)
df = df.append(df1,ignore_index=True)
df1
"""

"\n#last checked 3/22/20; still need to check sold out case\nnames = []; prices = []; saleprices = []; soldouts = []; stores = []; species = []\n\n#Cook's Carnivorous Plants\nURL = 'http://www.flytraps.com/Scripts/'\npage = urlopen(URL)\nsoup = BeautifulSoup(page,'html.parser')\nlinks = soup.findAll('span',attrs={'class':'CPcatDescProd'})\nlinks = links[:25] #from aldrovanda to nepenthes hybrid lowland\n\nfor link in links:\n    #Checks what type of species\n    kind = link.text\n    specie = check_species(kind)\n        \n    #Create URL2 to be URL of species type\n    path = link.a['href']\n    URL2 = URL + path\n    \n    #wait random time between 0-5 seconds before scraping data\n    r = random.randint(0,5)\n    time.sleep(r)\n    \n    page = urlopen(URL2)\n    soup = BeautifulSoup(page,'html.parser')\n    containers = soup.findAll('div',attrs={'class':'prod-classic'})\n    containers_count = len(containers)\n    \n    #print(URL2)\n    #If is not blank, scrapes info. otherwise sk

In [8]:
"""
#last checked 3/22/20
names = []; prices = []; saleprices = []; soldouts = []; stores = []; species = []

#Carnivero
URL = 'https://www.carnivero.com'
page = urlopen(URL)
soup = BeautifulSoup(page,'html.parser')
links = soup.findAll('div',attrs={'class':'grid__item small--one-half medium-up--one-fifth slide-up-animation animated'})
links = [links[i] for i in range(10) if i!=1]  #all categories except beginner plants

#Looks at plant categories page
for link in links:            
    #Create URL2 to be URL of species type
    path = link.a['href']
    URL2 = URL + path
    
    #wait random time between 0-5 seconds before scraping data
    r = random.randint(0,5)
    time.sleep(r)
  
    page_next = True
    while page_next:
        page = urlopen(URL2)
        soup = BeautifulSoup(page,'html.parser')
        containers = soup.findAll('div',attrs={'class':'product grid__item medium-up--one-third small--one-half slide-up-animation animated'})
    
        for container in containers:
            name = container.find('div',attrs={'class':'product__title text-center'}).text.strip()
            specie = check_species(name)

            try:
                price = container.find('span',attrs={'class':'product__price'}).text.strip()
                separate_index = price.find('$')+1 #look for 1st instance of $
                price = price[separate_index:] #remove the all characters before and including the string '$'
                prices = prices + [price]
                saleprices = saleprices + ['']           
            except AttributeError:
                #grab original price
                price = container.find('s').text.strip()
                separate_index = price.find('$')+1 #look for 1st instance of $
                price = price[separate_index:] #remove the all characters before and including the string '$'
                prices = prices + [price]
            
                #grab new sales price
                price = container.find('span',attrs={'class':'product__price--on-sale'}).text.strip()
                separate_index = price.find('$')+1 #look for 1st instance of $
                price = price[separate_index:] #remove the all characters before and including the string '$'
                saleprices = saleprices + [price] 
            
            try:
                soldout = container.find('strong',attrs={'class':'sold-out-text'}).text.strip()
                soldouts = soldouts + [soldout.lower()]
            except AttributeError:
                soldouts = soldouts + ['']
            
            names = names + [name]
            stores = stores + ['Carnivero']
            species = species + [specie]
        
        #Updates the URL for next page; stops the while loop if there is no next page
        try:
            path = soup.find('span',attrs={'class':'next'})
            path = path.a['href']
            URL2 = 'https://www.carnivero.com' + path
        except AttributeError:
            page_next = False
        
        
d = {'Name':names,'Price':prices,'Sale Price':saleprices,'Sold Out':soldouts,'Store':stores,'Species':species}
df1 = pd.DataFrame(d)
df = df.append(df1,ignore_index=True)
df1
"""

"\n#last checked 3/22/20\nnames = []; prices = []; saleprices = []; soldouts = []; stores = []; species = []\n\n#Carnivero\nURL = 'https://www.carnivero.com'\npage = urlopen(URL)\nsoup = BeautifulSoup(page,'html.parser')\nlinks = soup.findAll('div',attrs={'class':'grid__item small--one-half medium-up--one-fifth slide-up-animation animated'})\nlinks = [links[i] for i in range(10) if i!=1]  #all categories except beginner plants\n\n#Looks at plant categories page\nfor link in links:            \n    #Create URL2 to be URL of species type\n    path = link.a['href']\n    URL2 = URL + path\n    \n    #wait random time between 0-5 seconds before scraping data\n    r = random.randint(0,5)\n    time.sleep(r)\n  \n    page_next = True\n    while page_next:\n        page = urlopen(URL2)\n        soup = BeautifulSoup(page,'html.parser')\n        containers = soup.findAll('div',attrs={'class':'product grid__item medium-up--one-third small--one-half slide-up-animation animated'})\n    \n        for

In [9]:
"""
#last checked 3/22/20
names = []; prices = []; saleprices = []; soldouts = []; stores = []; species = []

#Native Exotics
URL = 'https://nativeexoticsonline.com'
h = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
req = Request(URL,headers=h) 
page = urlopen(req).read() 
soup = BeautifulSoup(page,'html.parser')
links = soup.find('ul',attrs={'class':'dropdown'})
links = links.findAll('a')
pages_of_interest = [5] + list(range(10,15)) #All Neps + Cephs,Drosera,..., Utrics
links = [links[i] for i in pages_of_interest]

#Looks at plant categories page
for link in links:      
    #Create URL2 to be URL of species type
    path = link['href']
    URL2 = URL + path
    
    #wait random time between 0-5 seconds before scraping data
    r = random.randint(0,5)
    time.sleep(r)
    
    page_next = True
    while page_next:
        req = Request(URL2,headers=h) 
        page = urlopen(req).read() 
        soup = BeautifulSoup(page,'html.parser')
        container_big = soup.find('ul',attrs={'class':'products columns-4'})
        containers = container_big.findAll('li')
    
        for container in containers:
            name = container.find('h2').text.strip()
            specie = check_species(name)
        
            price_list = container.findAll('span',attrs={'class':'woocommerce-Price-amount amount'})
            #2 prices listed when on sale. 1st list element is original price, 2nd element is sales price
            if len(price_list) == 2:
                price = price_list[0].text.strip()
                price = price[2:]
                saleprice = price_list[1].text.strip()
                saleprice = saleprice[2:]             
            elif len(price_list) == 1:
                price = price_list[0].text.strip()
                price = price[2:]
                saleprice = ''
        
            try:
                soldout = container.find('span',attrs={'class':'soldout'}).text.strip()
                soldouts = soldouts + [soldout.lower()]
            except AttributeError:
                soldouts = soldouts + ['']
            
            names = names + [name]
            prices = prices + [price]
            stores = stores + ['Native Exotics']
            saleprices = saleprices + [saleprice]
            species = species + [specie]
            
        #Updates the URL for next page; stops the while loop if there is no next page
        try:
            URL2 = soup.find('a',attrs={'class':'next page-numbers'})
            URL2 = URL2['href']
        except TypeError:
            page_next = False
        
        
d = {'Name':names,'Price':prices,'Sale Price':saleprices,'Sold Out':soldouts,'Store':stores,'Species':species}
df1 = pd.DataFrame(d)
df = df.append(df1,ignore_index=True)
df1
"""

"\n#last checked 3/22/20\nnames = []; prices = []; saleprices = []; soldouts = []; stores = []; species = []\n\n#Native Exotics\nURL = 'https://nativeexoticsonline.com'\nh = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}\nreq = Request(URL,headers=h) \npage = urlopen(req).read() \nsoup = BeautifulSoup(page,'html.parser')\nlinks = soup.find('ul',attrs={'class':'dropdown'})\nlinks = links.findAll('a')\npages_of_interest = [5] + list(range(10,15)) #All Neps + Cephs,Drosera,..., Utrics\nlinks = [links[i] for i in pages_of_interest]\n\n#Looks at plant categories page\nfor link in links:      \n    #Create URL2 to be URL of species type\n    path = link['href']\n    URL2 = URL + path\n    \n    #wait random time between 0-5 seconds before scraping data\n    r = random.randint(0,5)\n    time.sleep(r)\n    \n    page_next = True\n    while page_next:\n        req = Request(URL2,headers=h) \n        page = urlopen(req).read()

In [10]:
"""
#last checked: 3/22/20
names = []; prices = []; saleprices = []; soldouts = []; stores = []; species = []

#Carnivorous Plant Nursery
URL = 'https://carnivorousplantnursery.com'
page = urlopen(URL)
soup = BeautifulSoup(page,'html.parser')
links = soup.findAll('article')
links = links[:9] #Only looks at VFT to buterworts

#Looks at plant categories page
for link in links:      
    #Create URL2 to be URL of species type
    path = link.find('figure')
    path = path.a['href']
    URL2 = URL + path   
    
    #wait random time between 0-5 seconds before scraping data
    r = random.randint(0,5)
    time.sleep(r)
    
    page = urlopen(URL2)
    soup = BeautifulSoup(page,'html.parser')
    containers = soup.findAll('article',attrs={'class':'productgrid--item imagestyle--cropped-small '})
    for container in containers:
        name = container.find('h2').text.strip()
        specie = check_species(name)
    
        price = container.find('div',attrs={'class':'price--main'}).text.strip()
        separate_index = price.find('$')+1 #look for 1st instance of $
        price = price[separate_index:] #remove the all characters before and including the string '$'
    
        names = names + [name]
        prices = prices + [price]
        stores = stores + ['Carnivorous Plant Nursery']
        saleprices = saleprices + [''] #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!change when on sale
        species = species + [specie]
        
        #looks for sold out items
        try:
            soldout = container.find('span',attrs={'class':'productitem--badge badge--soldout'}).text.strip()
            soldouts = soldouts + [soldout.lower()]
        except AttributeError:
            soldouts = soldouts + ['']
            

d = {'Name':names,'Price':prices,'Sale Price':saleprices,'Sold Out':soldouts,'Store':stores,'Species':species}
df1 = pd.DataFrame(d)
df = df.append(df1,ignore_index=True) 
df1
"""

"\n#last checked: 3/22/20\nnames = []; prices = []; saleprices = []; soldouts = []; stores = []; species = []\n\n#Carnivorous Plant Nursery\nURL = 'https://carnivorousplantnursery.com'\npage = urlopen(URL)\nsoup = BeautifulSoup(page,'html.parser')\nlinks = soup.findAll('article')\nlinks = links[:9] #Only looks at VFT to buterworts\n\n#Looks at plant categories page\nfor link in links:      \n    #Create URL2 to be URL of species type\n    path = link.find('figure')\n    path = path.a['href']\n    URL2 = URL + path   \n    \n    #wait random time between 0-5 seconds before scraping data\n    r = random.randint(0,5)\n    time.sleep(r)\n    \n    page = urlopen(URL2)\n    soup = BeautifulSoup(page,'html.parser')\n    containers = soup.findAll('article',attrs={'class':'productgrid--item imagestyle--cropped-small '})\n    for container in containers:\n        name = container.find('h2').text.strip()\n        specie = check_species(name)\n    \n        price = container.find('div',attrs={'cl

In [11]:
"""
#last checked: 3/22/20
names = []; prices = []; saleprices = []; soldouts = []; stores = []; species = []

#Bergen Water Gardens
#cannot extract next page button functionality; cannot extract how many pages total
#instead keeps going to next page until error
try:
    page_number = 1
    while page_number < 16:
        URL = 'https://bergenwatergardens.com/product-category/carnivorous-plants/?fwp_paged=%d' % page_number
        h = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
        req = Request(URL,headers=h) 
        page = urlopen(req).read() 
        soup = BeautifulSoup(page,'html.parser')
    
        #wait random time between 0-5 seconds before scraping data
        r = random.randint(0,5)
        time.sleep(r)
        
        containers = soup.find('ul',attrs={'class':'products columns-4'})
        containers = containers.findAll('li')        
        
        for container in containers:
            name = container.find('h2').text.strip()
            specie = check_species(name)

            price = container.find('span',attrs={'class':'price'}).text.strip()
            #if item is on sale, gets normal price and sale price
            try:
                container.find('span',attrs={'class':'onsale'}).text.strip() == 'Sale!'
    
                separate_index = price.find('$')+2 #from BEGINNING of string, look for 1st instance of $
                separate_index2 = price.find(' ') #from BEGINNING of string, look for 1st instance of space
                price_before = price[separate_index:separate_index2] #gets original price
        
                separate_index = price.rfind('$')+2 #from END of string, look for 1st instance of $
                price_after = price[separate_index:] #gets sale price
    
                price = price_before
                saleprice = price_after
            #else just get the normal price
            except AttributeError:
                if price.rfind('$') != 0: #cases when prices is a range like $X.XX - $X.XX
                    price = container.find('span',attrs={'class':'price'}).text.strip()
                    separate_index = price.find('$')+2 #from BEGINNING of string, look for 1st instance of $
                    separate_index2 = price.find(' ') #from BEGINNING of string, look for 1st instance of space before -
                    price = price[separate_index:separate_index2] #remove everything except price
                    saleprice = ''
                else:
                    price = container.find('span',attrs={'class':'price'}).text.strip()
                    price = price[2:] #removes '$ ' before price
                    saleprice = ''
    
            names = names + [name]
            prices = prices + [price]
            stores = stores + ['Bergen Water Gardens']
            saleprices = saleprices + [saleprice]
            species = species + [specie]        
        
            #looks for sold out items
            try:
                URL2 = container.a['href']
                req = Request(URL2,headers=h) 
                page = urlopen(req).read() 
                soup = BeautifulSoup(page,'html.parser')
                
                soldout = soup.find('p',attrs={'class':'stock out-of-stock'}).text.strip()
                soldouts = soldouts + [soldout.lower()]
            except AttributeError:
                soldouts = soldouts + ['']
                        
        #increase page_number for while loop
        page_number = page_number + 1
#if container is empty, pass- does nothing, 'pass' in python instead of 'continue'
except (AttributeError,IndexError):
    pass 
    

d = {'Name':names,'Price':prices,'Sale Price':saleprices,'Sold Out':soldouts,'Store':stores,'Species':species}
df1 = pd.DataFrame(d)
df = df.append(df1,ignore_index=True) 
df1
"""

"\n#last checked: 3/22/20\nnames = []; prices = []; saleprices = []; soldouts = []; stores = []; species = []\n\n#Bergen Water Gardens\n#cannot extract next page button functionality; cannot extract how many pages total\n#instead keeps going to next page until error\ntry:\n    page_number = 1\n    while page_number < 16:\n        URL = 'https://bergenwatergardens.com/product-category/carnivorous-plants/?fwp_paged=%d' % page_number\n        h = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}\n        req = Request(URL,headers=h) \n        page = urlopen(req).read() \n        soup = BeautifulSoup(page,'html.parser')\n    \n        #wait random time between 0-5 seconds before scraping data\n        r = random.randint(0,5)\n        time.sleep(r)\n        \n        containers = soup.find('ul',attrs={'class':'products columns-4'})\n        containers = containers.findAll('li')        \n        \n        for container in con

In [12]:
"""
#last checked: 3/22/20; still need to check sold out case
names = []; prices = []; saleprices = []; soldouts = []; stores = []; species = []

#Petflytrap
#cannot extract next page button functionality; cannot extract how many pages total
#instead, manually provide page numbers to for-loop over
for page_number in range(1,16):
    URL = 'https://www.petflytrap.com/All-carnivorous-plants_c_11-%d-3.html' % page_number
    h = {'User-Agent':'Mozilla/5.0'}
    req = Request(URL,headers=h) 
    page = urlopen(req).read()
    soup = BeautifulSoup(page,'html.parser')

    #wait random time between 0-5 seconds before scraping data
    r = random.randint(0,5)
    time.sleep(r)

    containers = soup.find('div',attrs={'class':'product-items product-items-4'})

    for i in range(len(containers.findAll('div',attrs={'class':'name'}))):
        name = containers.findAll('div',attrs={'class':'name'})[i].text.strip()
        specie = check_species(name)
    
        price = containers.findAll('div',attrs={'class':'price'})[i].text.strip()
        #if item is on sale, seperate into price_before and price_after
        if price.rfind('$') != 0:
            separate_index = price.find('$')+1 #from BEGINNING of string, look for 1st instance of $
            separate_index2 = price.find('\n') #from BEGINNING of string, look for 1st instance of \n
            price_before = price[separate_index:separate_index2] #remove everything except price
        
            separate_index = price.rfind('$')+1 #from END of string, look for 1st instance of $
            separate_index2 = price.rfind('\n') #from END of string, look for 1st instance of \n
            price_after = price[separate_index:separate_index2] #remove everything except price
        else:
            price_before = price[1:] #removes '$' before price
            price_after = ''        


        names = names + [name]
        prices = prices + [price_before]
        saleprices = saleprices + [price_after]
        stores = stores + ['Petflytap']
        soldouts = soldouts + ['']
        species = species + [specie]

        
d = {'Name':names,'Price':prices,'Sale Price':saleprices,'Sold Out':soldouts,'Store':stores,'Species':species}
df1 = pd.DataFrame(d)
df = df.append(df1,ignore_index=True)
df1
"""

"\n#last checked: 3/22/20; still need to check sold out case\nnames = []; prices = []; saleprices = []; soldouts = []; stores = []; species = []\n\n#Petflytrap\n#cannot extract next page button functionality; cannot extract how many pages total\n#instead, manually provide page numbers to for-loop over\nfor page_number in range(1,16):\n    URL = 'https://www.petflytrap.com/All-carnivorous-plants_c_11-%d-3.html' % page_number\n    h = {'User-Agent':'Mozilla/5.0'}\n    req = Request(URL,headers=h) \n    page = urlopen(req).read()\n    soup = BeautifulSoup(page,'html.parser')\n\n    #wait random time between 0-5 seconds before scraping data\n    r = random.randint(0,5)\n    time.sleep(r)\n\n    containers = soup.find('div',attrs={'class':'product-items product-items-4'})\n\n    for i in range(len(containers.findAll('div',attrs={'class':'name'}))):\n        name = containers.findAll('div',attrs={'class':'name'})[i].text.strip()\n        specie = check_species(name)\n    \n        price = co

In [13]:
"""
#Predatory Plants
"""

'\n#Predatory Plants\n'

In [14]:
#Export concatinated df to Excel
date = '2020_03_28' #date format 2020_01_01
file_name_1 = 'df_%s.xlsx' % date
#df.to_excel(file_name_1)


#Converting exported df back into correct format
df = pd.read_excel(file_name_1)
#df = df.drop(columns='Unnamed: 0') #removing column of indecies

df


,Name,Price,Sale Price,Sold Out,Store,Species
0,72+ Hour Heat Pack - For shipping with plants,4.00,NaN,NaN,Pearl River Exotics,other
1,"Nepenthes ventricosa ""Red""",10.00,NaN,NaN,Pearl River Exotics,nepenthes
2,"Nepenthes robcantleyi ""Queen of Hearts"" x ""Kin...",149.00,NaN,NaN,Pearl River Exotics,nepenthes
3,Spagmoss 100 gram briquette,7.00,NaN,NaN,Pearl River Exotics,other
4,Nepenthes copelandii BE-3046,19.00,NaN,NaN,Pearl River Exotics,nepenthes
5,Nepenthes veitchii x platychila BE-3213,29.00,NaN,NaN,Pearl River Exotics,nepenthes
6,Nepenthes aristolochioides x ventricosa BE-3447,14.00,NaN,NaN,Pearl River Exotics,nepenthes
7,Nepenthes burkei BE-3254,18.00,NaN,sold out,Pearl River Exotics,nepenthes
8,Nepenthes spathulata x spectablilis BE-3314 *S...,15.00,NaN,NaN,Pearl River Exotics,nepenthes
9,Nepenthes albomarginata BE-3004,29.00,NaN,NaN,Pearl River Exotics,nepenthes


In [15]:
#Once the complete database is compiled, apply tagging function - tags any species found in product
#https://www.carnivorousplants.co.uk/resources/nepenthes-interactive-guide/
#https://en.wikipedia.org/wiki/List_of_Nepenthes_species
#https://en.wikipedia.org/wiki/List_of_Utricularia_species

#compare product to database of species known
#if str in database, then species = str
df['Cross1'] = ''
df['Cross2'] = ''
df['Cross3'] = ''
df['Cross4'] = ''
df['Cross5'] = ''
df['Cross6'] = ''
df['Cross7'] = ''

In [27]:
n = 175
name_nepenthes = [''] * n
URL = 'https://en.wikipedia.org/wiki/List_of_Nepenthes_species'
page = urlopen(URL)
soup = BeautifulSoup(page,'html.parser')
containers = soup.findAll('tr')

for i in range(1,n):
    container = containers[i]
    name = container.find('a')
    name_nepenthes[i] = name['title'][10:] #removing 'Nepenthes ' from string
    
name_nepenthes = name_nepenthes[1:] #remove header row

#lady = lady luck, bill = bill bailey
name_nepenthes_hybrids = ['lady','hookeriana','suki','bill','diana','briggsiana','rokko','ventrata', \
                          'miranda','trusmadiensis','gentle']
name_nepenthes_hybrids_cross = [['ventricosa','ampullaria'],['ampullaria','rafflesiana'], \
                                ['rafflesiana','sibuyanensis'],['ventricosa','singalana'], \
                                ['ventricosa','sibuyanensis','ampullaria'],['ventricosa','lowii'], \
                                ['maxima','thorelii'],['ventricosa','alata'],['maxima','northiana'], \
                                ['lowii','macrophylla'],['fusca','maxima']]

#name_nepenthes

In [29]:
n = 198
name_drosera = [''] * n
URL = 'https://en.wikipedia.org/wiki/List_of_Drosera_species'
page = urlopen(URL)
soup = BeautifulSoup(page,'html.parser')
containers = soup.findAll('tr')

for i in range(1,n):
    container = containers[i]
    try:
        name = container.a.text.strip()
        index = name.find('\xa0')+1 #look for 1st instance of '\xa0'
        name_drosera[i] = name[index:]
    except AttributeError:
        name = container.i.text.strip()
        index = name.find(' ')+1 #look for 1st instance of space
        name_drosera[i] = name[index:]
        
    
name_drosera = name_drosera[2:] #remove header rows

#pink = dork's pink, marston = Marston Dragon, sunset = California Sunset, butterfly = Butterfly Valley, 
#
name_drosera_hybrids = ['hercules','pink','dichotoma','multifida','marston','sunset','butterfly','andromeda', \
                        'indumenta','tracyii','dilatopetiolaris']
name_drosera_hybrids_cross = [['capensis','aliciae'],['callistos','lasiantha'],['binata'],['binata'],['binata'], \
                              ['filiformis'],['intermedia','filiformis'],['schizandra','prolifera'],['macrantha'], \
                              ['filiformis'],['dilatato–petiolaris']]

#name_drosera

In [31]:
name_utricularia = []
URL = 'https://en.wikipedia.org/wiki/List_of_Utricularia_species'
page = urlopen(URL)
soup = BeautifulSoup(page,'html.parser')
containers = soup.findAll('ul')

#scraping species names from containers[4] to containers[39]
for i in range(4,40):
    container = containers[i]
    string_names = container.text.replace('Utricularia ','').split('\n') #removing 'Utricularia ' from string

    #Cleaning up Cross names
    for j in range(len(string_names)):
        #Removing Footnotes [1]
        if '[' in string_names[j]:
            index = string_names[j].find('[')
            string_names[j] = string_names[j][:(index-1)]

            
    name_utricularia = name_utricularia + string_names

    
##have not found a natural hybrid yet; no need for this section yet
#name_utricularia_hybrids = ['lady','hookeriana','suki','bill','diana','briggsiana','rokko','ventrata', \
#                          'miranda','trusmadiensis']
#name_utricularia_hybrids_cross = [['ventricosa','ampullaria'],['ampullaria','rafflesiana'], \
#                                ['rafflesiana','sibuyanensis'],['ventricosa','singalana'], \
#                                ['ventricosa','sibuyanensis','ampullaria'],['ventricosa','lowii'], \
#                                ['maxima','thorelii'],['ventricosa','alata'],['maxima','northiana'], \
#                                ['lowii','macrophylla']]

#name_utricularia

In [33]:
name_pinguicula = []
URL = 'https://en.wikipedia.org/wiki/List_of_Pinguicula_species'
page = urlopen(URL)
soup = BeautifulSoup(page,'html.parser')
containers = soup.findAll('ul')


#scraping species names from containers[4] to containers[26]
for i in range(4,27):
    container = containers[i]
    string_names = container.text.replace('Pinguicula ','').split('\n') #removing 'Pinguicula ' from string
    
    #Cleaning up Cross names
    for j in range(len(string_names)):
        #Removing text after -
        if '–' in string_names[j]:
            index = string_names[j].find('–')
            string_names[j] = string_names[j][:(index-1)]
        #Removing Footnotes [1]
        elif '[' in string_names[j]:
            index = string_names[j].find('[')
            string_names[j] = string_names[j][:(index-1)]
            
            
    name_pinguicula = name_pinguicula + string_names
    

#john = john rizzi, alfred = alfred lau, 
name_pinguicula_hybrids = ['sethos','weser','john','nivalis','alfred','florian','gina','apasionada']
name_pinguicula_hybrids_cross = [['ehlersiae','moranensis'],['moranensis','ehlersiae'],['moranensis','?'],['nivalis'], \
                                 ['gigantea','?'],['debbertiana','esseriana'],['zecheri','agnata'],['gigantea','moctezumae']]

#name_pinguicula

In [34]:
#i is row index, j is column index
#len(df) gives number of rows in df
for i in range(len(df)):
    #removing all of the special characters:()[] so the string is split into 'ventricosa' and not '(ventricosa''
    string_split = df.at[i,'Name'].replace('(','').replace(')','').replace('[','').replace(']','').replace(',','')
    string_split = string_split.replace('-','').replace('–','').replace('\'','').replace('\"','').replace('‘','')
    string_split = string_split.replace('’','')
    #convert entire string to lowercase
    string_split = string_split.lower()
    
    #Filling in Cross columns
    string_split = string_split.split(' ')
    count = 1 #counter for column name
    repeat_cross = []
    
    #If-Else If by species
    if df.at[i,'Species'] == 'nepenthes':
        for j in range(len(string_split)):
            #if cross is in name_nepenthes and not repeated, add it to CrossX column
            if (string_split[j] in name_nepenthes) and (string_split[j] not in repeat_cross):
                column = 'Cross%d' % count
                count = count + 1
                df.loc[[i],[column]] = string_split[j]
                repeat_cross = repeat_cross + [string_split[j]]
            #if cross is in name_nepenthes_hybrid and not repeated, add it to CrossX column
            elif (string_split[j] in name_nepenthes_hybrids) and (string_split[j] not in repeat_cross):
                m = name_nepenthes_hybrids.index(string_split[j])
                for k in range(len(name_nepenthes_hybrids_cross[m])):
                    column = 'Cross%d' % count
                    count = count + 1
                    df.loc[[i],[column]] = name_nepenthes_hybrids_cross[m][k]
                    repeat_cross = repeat_cross + [string_split[j]]
                    repeat_cross = repeat_cross + [name_nepenthes_hybrids_cross[m][k]]
    elif df.at[i,'Species'] == 'drosera':
        for j in range(len(string_split)):
            #if cross is in name_drosera and not repeated, add it to CrossX column
            if (string_split[j] in name_drosera) and (string_split[j] not in repeat_cross):
                column = 'Cross%d' % count
                count = count + 1
                df.loc[[i],[column]] = string_split[j]
                repeat_cross = repeat_cross + [string_split[j]]
            #if cross is in name_drosera_hybrid and not repeated, add it to CrossX column
            elif (string_split[j] in name_drosera_hybrids) and (string_split[j] not in repeat_cross):
                m = name_drosera_hybrids.index(string_split[j])
                for k in range(len(name_drosera_hybrids_cross[m])):
                    column = 'Cross%d' % count
                    count = count + 1
                    df.loc[[i],[column]] = name_drosera_hybrids_cross[m][k]
                    repeat_cross = repeat_cross + [string_split[j]]
                    repeat_cross = repeat_cross + [name_drosera_hybrids_cross[m][k]]
    elif df.at[i,'Species'] == 'utricularia':
        for j in range(len(string_split)):
            #if cross is in name_utricularia and not repeated, add it to CrossX column
            if (string_split[j] in name_utricularia) and (string_split[j] not in repeat_cross):
                column = 'Cross%d' % count
                count = count + 1
                df.loc[[i],[column]] = string_split[j]
                repeat_cross = repeat_cross + [string_split[j]]  
    elif df.at[i,'Species'] == 'pinguicula':
        for j in range(len(string_split)):
            #if cross is in name_pinguicula and not repeated, add it to CrossX column
            if (string_split[j] in name_pinguicula) and (string_split[j] not in repeat_cross):
                column = 'Cross%d' % count
                count = count + 1
                df.loc[[i],[column]] = string_split[j]
                repeat_cross = repeat_cross + [string_split[j]]
            #if cross is in name_pinguicula_hybrid and not repeated, add it to CrossX column
            elif (string_split[j] in name_pinguicula_hybrids) and (string_split[j] not in repeat_cross):
                m = name_pinguicula_hybrids.index(string_split[j])
                for k in range(len(name_pinguicula_hybrids_cross[m])):
                    column = 'Cross%d' % count
                    count = count + 1
                    df.loc[[i],[column]] = name_pinguicula_hybrids_cross[m][k]
                    repeat_cross = repeat_cross + [string_split[j]]
                    repeat_cross = repeat_cross + [name_pinguicula_hybrids_cross[m][k]]
            
            


#create special if clause when tag is not found, open product page and scan description text for tags
df

,Name,Price,Sale Price,Sold Out,Store,Species,Cross1,Cross2,Cross3,Cross4,Cross5,Cross6,Cross7
0,72+ Hour Heat Pack - For shipping with plants,4.00,NaN,NaN,Pearl River Exotics,other,,,,,,,
1,"Nepenthes ventricosa ""Red""",10.00,NaN,NaN,Pearl River Exotics,nepenthes,ventricosa,,,,,,
2,"Nepenthes robcantleyi ""Queen of Hearts"" x ""Kin...",149.00,NaN,NaN,Pearl River Exotics,nepenthes,robcantleyi,,,,,,
3,Spagmoss 100 gram briquette,7.00,NaN,NaN,Pearl River Exotics,other,,,,,,,
4,Nepenthes copelandii BE-3046,19.00,NaN,NaN,Pearl River Exotics,nepenthes,copelandii,,,,,,
5,Nepenthes veitchii x platychila BE-3213,29.00,NaN,NaN,Pearl River Exotics,nepenthes,veitchii,platychila,,,,,
6,Nepenthes aristolochioides x ventricosa BE-3447,14.00,NaN,NaN,Pearl River Exotics,nepenthes,aristolochioides,ventricosa,,,,,
7,Nepenthes burkei BE-3254,18.00,NaN,sold out,Pearl River Exotics,nepenthes,burkei,,,,,,
8,Nepenthes spathulata x spectablilis BE-3314 *S...,15.00,NaN,NaN,Pearl River Exotics,nepenthes,spathulata,,,,,,
9,Nepenthes albomarginata BE-3004,29.00,NaN,NaN,Pearl River Exotics,nepenthes,albomarginata,,,,,,


In [38]:
"""
#Check what does not have any Cross1 tag
for i in range(len(df)):
    if (df.loc[i,'Cross1'] is '') and (df.loc[i,'Species'] == 'pinguicula'):
        print(i,df.loc[i,'Name'])
"""

"\n#Check what does not have any Cross1 tag\nfor i in range(len(df)):\n    if (df.loc[i,'Cross1'] is '') and (df.loc[i,'Species'] == 'pinguicula'):\n        print(i,df.loc[i,'Name'])\n"

In [24]:
#%whos

In [37]:
"""
#Export df with Cross1, ... ,Cross6 to Excel
file_name_2 = 'dfCross_%s.xlsx' % date
df.to_excel(file_name_2)


#Converting exported df back into correct format
df = pd.read_excel('dfCross_2020_02_05.xlsx')
df = df.drop(columns='Unnamed: 0') #removing column of indecies

df
"""

"\n\n#Converting exported df back into correct format\ndf = pd.read_excel('dfCross_2020_02_05.xlsx')\ndf = df.drop(columns='Unnamed: 0') #removing column of indecies\n\ndf\n"